In [1]:
from joblib import load, dump
import pandas as pd
import numpy as np
import os
import itertools

In [16]:
out1 = load('../output/output/pipeline_multiples_100-150_debug_new.joblib')

In [17]:
print(len(out1))
print(len(out1[0]))
print(out1[0][0])
print(len(out1[0][1]))
print(out1[0][1].head())
print(out1[0][2])
print(out1[0][3])

print(len(out1[39]))
print(out1[39][0])
print(len(out1[39][1]))
print(out1[39][1].head())
print(out1[39][2])
print(out1[39][3])

50
4
(37.89915966386555, 5.411764705882353, 0.33865546218487397, 0.8184873949579833, 0.61)
4975
   TSNE_1_pipeline_0  TSNE_2_pipeline_0
0           6.688230          -2.030429
1          -5.398230          -2.593120
2          -5.465337          -2.613461
3         -10.151259           3.356815
4          -2.392145         -11.869175
156.55978226661682
2.618137417015028
4
(17.18487394957983, 27.294117647058822, 0.4563025210084034, 0.8033613445378152, 0.64)
4975
   TSNE_1_pipeline_39  TSNE_2_pipeline_39
0           15.801698           20.022610
1          -20.133116            3.886482
2          -20.463723            4.486366
3          -15.912435           -4.457298
4          -13.997911           14.277308
151.14372944831848
2.9064790678134216


In [3]:
def extract_data_from_file(output_file_path):
  
  output = load(output_file_path)
  
  combinations = [tuple[0] for tuple in output]
  #affinity_runtime_min = [output[1]/60 for tuple in output[0]]
  tsne_runtime_min = [tuple[2]/60 for tuple in output]
  KL_divergences = [tuple[3] for tuple in output]
  #Pipeline_indices = [tuple[3] for tuple in output]

  modified_tsne_dfs = []
  for tuple in output:
    combination = tuple[0]
    df_tsne = tuple[1]
    df_tsne.columns = [f'tSNE1_{combination}', f'tSNE2_{combination}']
    modified_tsne_dfs.append(df_tsne)

  merged_tsnes_from_file = pd.concat(modified_tsne_dfs, axis=1)

  perplexity = [comb[0] for comb in combinations] # 5
  early_exaggeration = [comb[1] for comb in combinations]
  initial_momentum = [comb[2] for comb in combinations]
  final_momentum = [comb[3] for comb in combinations]
  theta = [comb[4] for comb in combinations]

  pipeline_metrics = pd.DataFrame({
                                 'Combination' : combinations,
                                 'Perplexity': perplexity,
                                 'Early_exaggeration': early_exaggeration,
                                 'Initial_momentum': initial_momentum,
                                 'Final_momentum': final_momentum,
                                 'Theta': theta,
                                 'tSNE_runtime_min': tsne_runtime_min,
                                 'KL_divergence': KL_divergences
                                 })

  return(merged_tsnes_from_file, pipeline_metrics)


In [4]:
# testing
out1 = extract_data_from_file('../output/output/pipeline_multiples_0-10_debug_new.joblib')
print(out1[0].shape)
print(out1[0].columns[:5])
print(out1[1][:5])
#output = load('/home/luana/workspace/output/thesis/perp5/pipeline_multiples_perp5_0-60.joblib')
#print(len(output[0]))

(4975, 20)
Index(['tSNE1_(63.487394957983184, 7.0588235294117645, 0.2008403361344538, 0.8, 0.12)',
       'tSNE2_(63.487394957983184, 7.0588235294117645, 0.2008403361344538, 0.8, 0.12)',
       'tSNE1_(75.67226890756302, 10.117647058823529, 0.35546218487394965, 0.8537815126050421, 0.4)',
       'tSNE2_(75.67226890756302, 10.117647058823529, 0.35546218487394965, 0.8537815126050421, 0.4)',
       'tSNE1_(43.99159663865546, 4.9411764705882355, 0.44285714285714284, 0.8672268907563025, 0.88)'],
      dtype='object')
                                         Combination  Perplexity  \
0  (63.487394957983184, 7.0588235294117645, 0.200...   63.487395   
1  (75.67226890756302, 10.117647058823529, 0.3554...   75.672269   
2  (43.99159663865546, 4.9411764705882355, 0.4428...   43.991597   
3  (25.71428571428571, 4.235294117647059, 0.35546...   25.714286   
4  (34.24369747899159, 20.470588235294116, 0.3319...   34.243697   

   Early_exaggeration  Initial_momentum  Final_momentum  Theta  \
0       

In [19]:
df_maps = out1[0]
#print(df_test1.head)
df_metrics_test = out1[1]
unique_kl, ind_unique_kl = np.unique(df_metrics_test['KL_divergence'], return_index=True)
print(len(ind_unique_kl))

10


In [5]:
def merge_output_from_folder(output_base_path, folder_relative_path):
  output_files = os.listdir(os.path.join(output_base_path, folder_relative_path))
  tsne_maps_list = []
  pipeline_metrics_list = []
  for file in output_files:
    if file != 'README':
      full_file_path = os.path.join(output_base_path, folder_relative_path, file)
      data_from_file = extract_data_from_file(full_file_path)
      tsne_maps_list.append(data_from_file[0])
      pipeline_metrics_list.append(data_from_file[1])
     
  return(tsne_maps_list, pipeline_metrics_list)

In [14]:
# Extract data for all folders
output_path = 'C:/Users/luana/Documents/thesis_python/output'
#folders = [dir for dir in os.listdir(output_path) if os.path.isdir(dir)] #os.listdir(output_path)
folder = 'output'

#all_tsne_maps = []
#all_pipeline_metrics = []

#for folder in folders:
   #output_folder = merge_output_from_folder(output_base_path=output_path, folder_relative_path=folder)
   #all_tsne_maps.append(output_folder[0])
   #all_pipeline_metrics.append(output_folder[1])


output_folder = merge_output_from_folder(output_base_path=output_path, folder_relative_path=folder)
#all_tsne_maps = [tsne_df for tsne_df in itertools.chain(*all_tsne_maps)]
#all_pipeline_metrics = [metrics_df for metrics_df in itertools.chain(*all_pipeline_metrics)]



ValueError: No objects to concatenate

In [91]:
output_base_path = 'C:/Users/luana/Documents/thesis_python/output'
folder_relative_path = 'output'
output_files = os.listdir(os.path.join(output_base_path, folder_relative_path))
output_files = sorted(output_files)
print(len(output_files))
tsne_maps_list = []
pipeline_metrics_list = []
full_file_paths = []
#output_files = ['C:\\Users\\luana\\Documents\\thesis_python\\output\\output\\pipeline_multiples_0-10_debug_new.joblib']
for file in output_files[:13]:
    #if file != 'README':
    full_file_path = os.path.normpath(os.path.join(output_base_path, folder_relative_path, file))
    ## for windows!!
    #full_file_path = full_file_path.replace('\\', '\\\\')
    full_file_paths.append(full_file_path)
    #print(os.path.exists(full_file_path))
    data_from_file = extract_data_from_file(full_file_path)
    print(f"Processed file: {full_file_path}")
    tsne_maps_list.append(data_from_file[0])
    pipeline_metrics_list.append(data_from_file[1])

#tsne_maps = [tsne_df for tsne_df in itertools.chain(*all_tsne_maps)]
#all_pipeline_metrics = [metrics_df for metrics_df in itertools.chain(*all_pipeline_metrics)]
print(len(tsne_maps_list))
print(len(tsne_maps_list[0]))
print(len(pipeline_metrics_list))

15
Processed file: C:\Users\luana\Documents\thesis_python\output\output\pipeline_multiples_0-10_debug_new.joblib
Processed file: C:\Users\luana\Documents\thesis_python\output\output\pipeline_multiples_10-30_debug_new.joblib
Processed file: C:\Users\luana\Documents\thesis_python\output\output\pipeline_multiples_100-150_debug_new.joblib
Processed file: C:\Users\luana\Documents\thesis_python\output\output\pipeline_multiples_150-200_debug_new.joblib
Processed file: C:\Users\luana\Documents\thesis_python\output\output\pipeline_multiples_200-250_debug_new.joblib
Processed file: C:\Users\luana\Documents\thesis_python\output\output\pipeline_multiples_250-300_debug_new.joblib
Processed file: C:\Users\luana\Documents\thesis_python\output\output\pipeline_multiples_30-60_debug_new.joblib
Processed file: C:\Users\luana\Documents\thesis_python\output\output\pipeline_multiples_300-350_debug_new.joblib
Processed file: C:\Users\luana\Documents\thesis_python\output\output\pipeline_multiples_350-400_debu

In [86]:
print(len(full_file_paths))
#print(full_file_paths)
#print(full_file_paths[-2])
print(len(output_files) - len(full_file_paths))
print(output_files[13:])
#print(len(data_from_file))
#data = extract_data_from_file('../output/output/' + output_files[13])
## REDO FOR THESE TWO!

13
2
['pipeline_multiples_550-580_debug_new.joblib', 'pipeline_multiples_60-100_debug_new.joblib']


In [88]:
#nbr_files_folder = [len(os.listdir(os.path.join(output_path, folder))) for folder in folders]
#print(len(tsne_maps_list))

13


In [92]:
df_tsne_all = pd.concat(tsne_maps_list, axis = 1)
df_metrics_all = pd.concat(pipeline_metrics_list, axis = 0)

In [93]:
# checking dimensions
print(2*df_metrics_all.shape[0] == df_tsne_all.shape[1])

True


In [94]:
df_tsne_all

,"tSNE1_(63.487394957983184, 7.0588235294117645, 0.2008403361344538, 0.8, 0.12)","tSNE2_(63.487394957983184, 7.0588235294117645, 0.2008403361344538, 0.8, 0.12)","tSNE1_(75.67226890756302, 10.117647058823529, 0.35546218487394965, 0.8537815126050421, 0.4)","tSNE2_(75.67226890756302, 10.117647058823529, 0.35546218487394965, 0.8537815126050421, 0.4)","tSNE1_(43.99159663865546, 4.9411764705882355, 0.44285714285714284, 0.8672268907563025, 0.88)","tSNE2_(43.99159663865546, 4.9411764705882355, 0.44285714285714284, 0.8672268907563025, 0.88)","tSNE1_(25.71428571428571, 4.235294117647059, 0.35546218487394965, 0.8050420168067227, 0.4)","tSNE2_(25.71428571428571, 4.235294117647059, 0.35546218487394965, 0.8050420168067227, 0.4)","tSNE1_(34.24369747899159, 20.470588235294116, 0.33193277310924374, 0.9949579831932773, 0.43)","tSNE2_(34.24369747899159, 20.470588235294116, 0.33193277310924374, 0.9949579831932773, 0.43)",...,"tSNE1_(117.10084033613444, 10.823529411764707, 0.23109243697478993, 0.8302521008403362, 0.13)","tSNE2_(117.10084033613444, 10.823529411764707, 0.23109243697478993, 0.8302521008403362, 0.13)","tSNE1_(123.19327731092436, 20.941176470588236, 0.31848739495798317, 0.9949579831932773, 0.21)","tSNE2_(123.19327731092436, 20.941176470588236, 0.31848739495798317, 0.9949579831932773, 0.21)","tSNE1_(84.2016806722689, 20.235294117647058, 0.48991596638655466, 0.9546218487394958, 0.23)","tSNE2_(84.2016806722689, 20.235294117647058, 0.48991596638655466, 0.9546218487394958, 0.23)","tSNE1_(131.72268907563023, 27.058823529411764, 0.25798319327731095, 0.9327731092436975, 0.15)","tSNE2_(131.72268907563023, 27.058823529411764, 0.25798319327731095, 0.9327731092436975, 0.15)","tSNE1_(120.7563025210084, 30.352941176470587, 0.24453781512605044, 0.9697478991596639, 0.29)","tSNE2_(120.7563025210084, 30.352941176470587, 0.24453781512605044, 0.9697478991596639, 0.29)"
0,5.193212,-4.279599,-3.792464,-4.038485,-8.391545,-0.727661,6.260917,-9.228146,970.554033,1489.252817,...,4.344013,-0.628806,-720.630267,-839.310438,-8.330457,7.588347,2.598436,-3.597000,43.933079,-124.805827
1,-0.212047,-0.445417,-0.757104,-0.253508,-0.934909,-3.858821,-14.200690,11.351874,270.289255,-431.414421,...,0.765846,-0.277850,-1087.655340,1409.854873,0.876520,6.775179,0.234278,-1.034237,-54.532177,-37.084668
2,-0.269999,-0.501903,-0.753229,-0.360467,-1.076626,-3.877158,-13.718072,11.898977,355.439187,-389.183293,...,0.741869,-0.286985,-1099.349709,1370.564527,0.750374,6.538322,0.243699,-1.056942,-59.102032,-37.911469
3,2.964016,4.703401,4.911582,1.529277,5.190236,-4.947864,-3.039184,17.572859,171.376715,-774.688600,...,-3.334421,-1.340109,-1911.149046,1001.800518,14.664162,0.907986,1.011960,2.929705,-106.463121,151.836984
4,-4.118650,-6.405406,2.764997,-4.836288,-5.058496,-3.338548,-19.580391,-8.109108,-1295.560808,-1008.355607,...,2.115964,-3.646348,-1080.376527,-24.057970,-7.118078,-1.050444,0.900967,-2.128101,87.979247,-3.611139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4970,-7.116107,-4.962727,-5.735017,1.210349,-7.538346,-4.573368,2.865862,4.489933,1168.169753,421.343158,...,-0.447689,-4.932625,-1623.246523,1098.462488,9.102413,0.203900,-4.324875,-2.832222,37.740765,-190.368765
4971,-1.037164,-1.240234,-0.035401,-0.723275,-1.597613,-1.974645,-8.700048,1.726474,-1245.333635,997.987114,...,1.020311,0.126147,-1024.523670,1067.267386,-0.146677,6.005828,0.918813,-0.867610,-54.898871,-25.578265
4972,-1.193705,-1.872983,-0.062545,-1.339029,-1.814150,-1.538203,-4.627990,-3.324295,-1479.208824,1243.828324,...,1.158621,-0.459839,-1105.398857,1356.622792,-1.553493,4.516164,1.876763,-1.357837,-26.746367,16.354923
4973,-1.276572,-1.934101,0.254216,-1.268503,-1.636729,-1.446842,-8.196914,1.109377,-1331.736587,-962.782551,...,0.555039,-0.593826,-1384.557990,1387.092431,-0.079788,3.878217,0.567258,-1.135542,-28.908973,28.569086


In [95]:
df_metrics_all

,Combination,Perplexity,Early_exaggeration,Initial_momentum,Final_momentum,Theta,tSNE_runtime_min,KL_divergence
0,"(63.487394957983184, 7.0588235294117645, 0.200...",63.487395,7.058824,0.200840,0.800000,0.12,20.594587,2.436188
1,"(75.67226890756302, 10.117647058823529, 0.3554...",75.672269,10.117647,0.355462,0.853782,0.40,4.796836,2.361166
2,"(43.99159663865546, 4.9411764705882355, 0.4428...",43.991597,4.941176,0.442857,0.867227,0.88,1.720338,2.563719
3,"(25.71428571428571, 4.235294117647059, 0.35546...",25.714286,4.235294,0.355462,0.805042,0.40,4.145446,2.759805
4,"(34.24369747899159, 20.470588235294116, 0.3319...",34.243697,20.470588,0.331933,0.994958,0.43,4.269935,3.953748
...,...,...,...,...,...,...,...,...
30,"(117.10084033613444, 10.823529411764707, 0.231...",117.100840,10.823529,0.231092,0.830252,0.13,22.319422,2.152873
31,"(123.19327731092436, 20.941176470588236, 0.318...",123.193277,20.941176,0.318487,0.994958,0.21,11.803930,3.284151
32,"(84.2016806722689, 20.235294117647058, 0.48991...",84.201681,20.235294,0.489916,0.954622,0.23,10.900197,2.395314
33,"(131.72268907563023, 27.058823529411764, 0.257...",131.722689,27.058824,0.257983,0.932773,0.15,18.829109,2.129835


In [102]:
# duplicated pipelines
KL_divergence = df_metrics_all['KL_divergence'].to_numpy()
unique_values, indices_unique = np.unique(KL_divergence, return_index=True)
print(f'There are {len(KL_divergence) - len(unique_values)} duplicated pipelines')
#indices_duplicated = np.setdiff1d(np.arange(len(combinations)), indices_unique)
#print(combinations[indices_duplicated[0]])
#df_metrics_all[df_metrics_all['Combination'] == combinations[indices_duplicated[0]]]
#.isin(combinations[indices_duplicated[0]]))

There are 0 duplicated pipelines


In [98]:
#df_metrics_all.iloc[sorted(indices_unique)]

In [14]:
# checking if the filter is correct
#all(df_metrics_all.iloc[sorted(indices_unique)]['Combination'].to_numpy() == combinations[sorted(indices_unique)])

True

In [106]:
df_metrics_unique = df_metrics_all #.iloc[sorted(indices_unique)]
#.sort_values(by=['Perplexity', 'Perplexity', 'Early_exaggeration', 'Initial_momentum', 'Final_momentum', 'Theta'])
#df_metrics_unique.loc[:,'Pipeline_index'] = [f'pipeline_{i}' for i in np.arange(len(df_metrics_unique))]
#df_metrics_unique
df_metrics_unique

,Combination,Perplexity,Early_exaggeration,Initial_momentum,Final_momentum,Theta,tSNE_runtime_min,KL_divergence
0,"(63.487394957983184, 7.0588235294117645, 0.200...",63.487395,7.058824,0.200840,0.800000,0.12,20.594587,2.436188
1,"(75.67226890756302, 10.117647058823529, 0.3554...",75.672269,10.117647,0.355462,0.853782,0.40,4.796836,2.361166
2,"(43.99159663865546, 4.9411764705882355, 0.4428...",43.991597,4.941176,0.442857,0.867227,0.88,1.720338,2.563719
3,"(25.71428571428571, 4.235294117647059, 0.35546...",25.714286,4.235294,0.355462,0.805042,0.40,4.145446,2.759805
4,"(34.24369747899159, 20.470588235294116, 0.3319...",34.243697,20.470588,0.331933,0.994958,0.43,4.269935,3.953748
...,...,...,...,...,...,...,...,...
30,"(117.10084033613444, 10.823529411764707, 0.231...",117.100840,10.823529,0.231092,0.830252,0.13,22.319422,2.152873
31,"(123.19327731092436, 20.941176470588236, 0.318...",123.193277,20.941176,0.318487,0.994958,0.21,11.803930,3.284151
32,"(84.2016806722689, 20.235294117647058, 0.48991...",84.201681,20.235294,0.489916,0.954622,0.23,10.900197,2.395314
33,"(131.72268907563023, 27.058823529411764, 0.257...",131.722689,27.058824,0.257983,0.932773,0.15,18.829109,2.129835


In [101]:
#len(np.unique(df_metrics_unique['KL_divergence']))

In [100]:
#columns_tsne = df_tsne_all.columns.to_numpy()
#unique_values, indices_unique = np.unique(columns_tsne, return_index=True)
#print(f'There are {len(columns_tsne) - len(unique_values)} duplicated pipelines')
#print('The number of duplicated columns is twice the number of duplicated combinations:', len(columns_tsne) - len(unique_values) == 2*len(indices_duplicated))

In [104]:
df_tsne_unique = df_tsne_all #.iloc[:,sorted(indices_unique)]
df_tsne_unique

,"tSNE1_(63.487394957983184, 7.0588235294117645, 0.2008403361344538, 0.8, 0.12)","tSNE2_(63.487394957983184, 7.0588235294117645, 0.2008403361344538, 0.8, 0.12)","tSNE1_(75.67226890756302, 10.117647058823529, 0.35546218487394965, 0.8537815126050421, 0.4)","tSNE2_(75.67226890756302, 10.117647058823529, 0.35546218487394965, 0.8537815126050421, 0.4)","tSNE1_(43.99159663865546, 4.9411764705882355, 0.44285714285714284, 0.8672268907563025, 0.88)","tSNE2_(43.99159663865546, 4.9411764705882355, 0.44285714285714284, 0.8672268907563025, 0.88)","tSNE1_(25.71428571428571, 4.235294117647059, 0.35546218487394965, 0.8050420168067227, 0.4)","tSNE2_(25.71428571428571, 4.235294117647059, 0.35546218487394965, 0.8050420168067227, 0.4)","tSNE1_(34.24369747899159, 20.470588235294116, 0.33193277310924374, 0.9949579831932773, 0.43)","tSNE2_(34.24369747899159, 20.470588235294116, 0.33193277310924374, 0.9949579831932773, 0.43)",...,"tSNE1_(117.10084033613444, 10.823529411764707, 0.23109243697478993, 0.8302521008403362, 0.13)","tSNE2_(117.10084033613444, 10.823529411764707, 0.23109243697478993, 0.8302521008403362, 0.13)","tSNE1_(123.19327731092436, 20.941176470588236, 0.31848739495798317, 0.9949579831932773, 0.21)","tSNE2_(123.19327731092436, 20.941176470588236, 0.31848739495798317, 0.9949579831932773, 0.21)","tSNE1_(84.2016806722689, 20.235294117647058, 0.48991596638655466, 0.9546218487394958, 0.23)","tSNE2_(84.2016806722689, 20.235294117647058, 0.48991596638655466, 0.9546218487394958, 0.23)","tSNE1_(131.72268907563023, 27.058823529411764, 0.25798319327731095, 0.9327731092436975, 0.15)","tSNE2_(131.72268907563023, 27.058823529411764, 0.25798319327731095, 0.9327731092436975, 0.15)","tSNE1_(120.7563025210084, 30.352941176470587, 0.24453781512605044, 0.9697478991596639, 0.29)","tSNE2_(120.7563025210084, 30.352941176470587, 0.24453781512605044, 0.9697478991596639, 0.29)"
0,5.193212,-4.279599,-3.792464,-4.038485,-8.391545,-0.727661,6.260917,-9.228146,970.554033,1489.252817,...,4.344013,-0.628806,-720.630267,-839.310438,-8.330457,7.588347,2.598436,-3.597000,43.933079,-124.805827
1,-0.212047,-0.445417,-0.757104,-0.253508,-0.934909,-3.858821,-14.200690,11.351874,270.289255,-431.414421,...,0.765846,-0.277850,-1087.655340,1409.854873,0.876520,6.775179,0.234278,-1.034237,-54.532177,-37.084668
2,-0.269999,-0.501903,-0.753229,-0.360467,-1.076626,-3.877158,-13.718072,11.898977,355.439187,-389.183293,...,0.741869,-0.286985,-1099.349709,1370.564527,0.750374,6.538322,0.243699,-1.056942,-59.102032,-37.911469
3,2.964016,4.703401,4.911582,1.529277,5.190236,-4.947864,-3.039184,17.572859,171.376715,-774.688600,...,-3.334421,-1.340109,-1911.149046,1001.800518,14.664162,0.907986,1.011960,2.929705,-106.463121,151.836984
4,-4.118650,-6.405406,2.764997,-4.836288,-5.058496,-3.338548,-19.580391,-8.109108,-1295.560808,-1008.355607,...,2.115964,-3.646348,-1080.376527,-24.057970,-7.118078,-1.050444,0.900967,-2.128101,87.979247,-3.611139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4970,-7.116107,-4.962727,-5.735017,1.210349,-7.538346,-4.573368,2.865862,4.489933,1168.169753,421.343158,...,-0.447689,-4.932625,-1623.246523,1098.462488,9.102413,0.203900,-4.324875,-2.832222,37.740765,-190.368765
4971,-1.037164,-1.240234,-0.035401,-0.723275,-1.597613,-1.974645,-8.700048,1.726474,-1245.333635,997.987114,...,1.020311,0.126147,-1024.523670,1067.267386,-0.146677,6.005828,0.918813,-0.867610,-54.898871,-25.578265
4972,-1.193705,-1.872983,-0.062545,-1.339029,-1.814150,-1.538203,-4.627990,-3.324295,-1479.208824,1243.828324,...,1.158621,-0.459839,-1105.398857,1356.622792,-1.553493,4.516164,1.876763,-1.357837,-26.746367,16.354923
4973,-1.276572,-1.934101,0.254216,-1.268503,-1.636729,-1.446842,-8.196914,1.109377,-1331.736587,-962.782551,...,0.555039,-0.593826,-1384.557990,1387.092431,-0.079788,3.878217,0.567258,-1.135542,-28.908973,28.569086


In [107]:
# checking final dimensions
print(2*df_metrics_unique.shape[0] == df_tsne_unique.shape[1])

True


In [108]:
# Export csv
df_tsne_unique.to_csv('../output/df_tsne_momentum.csv', index=False, compression='gzip')
df_metrics_unique.to_csv('../output/df_metrics_momentum.csv', index=False)

In [21]:
# Load files
#df_tsne = pd.read_csv('../output/df_tsne_momentum.csv', compression='gzip')
#print(df_tsne.shape)

In [109]:
# Export joblib
dump(df_tsne_unique, '../output/df_tsne_momentum.joblib.gz', compress=('gzip', 3))
#dump(df_metrics_unique, '../output/df_metrics_momentum.joblib.gz', compress=('gzip', 3))

['../output/df_tsne_momentum.joblib.gz']

In [23]:
# Load filed
#df_tsne_unique_read = load('../output/df_tsne_unique.joblib.gz')
#print(df_tsne_unique_read.shape)